<a href="https://colab.research.google.com/github/MarcosRolando/OrgaDeDatosTP2/blob/main/neuralNetwoork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import math as mt
from tensorflow import feature_column


Tuneamos algunas cosas del DataFrame que notamos en el analisis de los datos.

In [2]:
def preprocess_dataframe(df):

  df = df[df['Stage'].isin(['Closed Won', 'Closed Lost'])]
  df.loc[:, 'Stage'].replace({'Closed Won':1, 'Closed Lost':0}, inplace=True) #0 corresponde a que el caso fue Closed Lost, 1 a que fue Closed Won. Asi tenemos un problema de clasificacion binario que puede entender la red neuronal.

  df.loc[:, 'Planned_Delivery_Start_Date'] = pd.to_datetime(df['Planned_Delivery_Start_Date'], 'coerce',
                                                                  format='%m/%d/%Y')
  df.loc[:, 'Planned_Delivery_End_Date'] = pd.to_datetime(df['Planned_Delivery_End_Date'], 'coerce',
                                                                                      format='%m/%d/%Y')
  df = df[df['Opportunity_ID'] != 9773] #Hardcodeo este filtrado porque el id 9773 tiene mal cargada la fecha de delviery end, dando una diferencia de 200 anios xd"

  #Pongo .loc porque pandas me jode con warnings que son falsos positivos de slice copy"
  #Gracias Pandas!"

  #Creamos una nueva columna (Feature Engineering) que contiene la longitud en dias 
  #estimada de la operacion. En el informe habiamos encontrado que aparentaba haber
  #una relacion cuadratica de decrecimiento a medida que aumentaban los dias donde disminuia
  #la chance de completar la operacion.
  df['Delta_Time'] = df['Planned_Delivery_End_Date'] - df['Planned_Delivery_Start_Date']
  df.loc[:, 'Delta_Time'] = df['Delta_Time'].dt.days

  #Pasamos todo a dolares
  currency_conversion = {'AUD':0.707612, 'EUR':1.131064, 'GBP':1.318055, 'JPY':0.008987, 'USD':1.0}
  df['Total_Taxable_Amount_Currency'] = df[['Total_Taxable_Amount_Currency']].replace(currency_conversion)
  df['Total_Taxable_Amount'] = df['Total_Taxable_Amount_Currency'] * df['Total_Taxable_Amount']

  #Borro columnas que tengan el mismo dato en todas las entradas, o inconsecuentes como el ID / Opportunity_ID
  #Algunas columnas borradas son porque pienso que no tienen incidencia, ir viendo.
  #TODO: Analizar si el Sales_Contract_No no es que importe el numero en si, sino si tiene
  #o no tiene numero de contrato. Por ahora no lo meto como input.
  #TODO: Ver el mismo tema con la columna 'Price', la mayoria tiene None u Other
  #y solo unos pocos tienen precio numerico. Quiza importe que tenga precio o no tenga,
  #o si no tiene precio quiza importe si es None u Other. Por ahora no lo pongo
  #como input.
  df.drop(columns=['Submitted_for_Approval', 'Last_Activity', 'ASP_(converted)_Currency', 
                  'Prod_Category_A', 'ID', 'Opportunity_ID'],inplace=True)

  #Renombramos las columnas que tienen caracteres que TensorFlow no acepta como validos.
  #Estos particularmente son whitespace, coma y parentesis por ejemplo.
  df.rename(columns={'ASP_(converted)':'ASP_converted','Pricing, Delivery_Terms_Quote_Appr':
                    'Pricing_Delivery_Terms_Quote_Appr','Pricing, Delivery_Terms_Approved':
                    'Pricing_Delivery_Terms_Approved','Source ':'Source'},inplace=True)

  #Drop columnas que quiza podamos usar pero por ahora no las uso
  df.drop(columns=['Account_Created_Date','Opportunity_Created_Date',
                  'Quote_Expiry_Date','Last_Modified_Date',
                  'Planned_Delivery_Start_Date','Planned_Delivery_End_Date',
                  'Month','Delivery_Quarter', 'Delivery_Year', 'Actual_Delivery_Date',
                  'Sales_Contract_No','Price','ASP','ASP_Currency','Total_Amount_Currency',
                  'Total_Amount','Total_Taxable_Amount_Currency','Currency'],
                   inplace=True)

  #Definimos que tipo de feature es cada columna

  #Debemos separar algunos de los registros para armar un set de test propio (no el de la catedra). De esta forma sabremos rapidamente
  #si nuestro modelo esta dando resultados optimos o no sin necesidad de estar subiendo el TP a Kaggle constantemente.
  #Sin embargo, no queremos usar tantos registros ya que estariamos disminuyendo el set de entrenamiento considerablemente.
  #Podemos empezar reservando 2000 registros para el test de prueba y ver que onda. Pasariamos de tener 16 mil a 14 mil 
  #registros para el set de entrenamiento, no es una perdida importantisima creo en principio, asi que arrancamos con eso.

  #Por otro lado, nuestro test de prueba deberia tener un 50 50 de Closed Won y Closed Lost, por lo que no podemos elegir asi nomas
  #al azar.

  #TODO: DEJO COMO TAREA ARMAR EL SET DE PRUEBA, POR AHORA SOLO NOS PREOCUPAMOS DE LOGRAR HACER ANDAR LA RED NEURONAL.
  normalized_columns = ['ASP_converted','TRF','Total_Taxable_Amount']
  for column in normalized_columns:
    df[column] = (df[column] - df[column].mean()) / df[column].std()

  df.fillna(value=0, inplace=True)

  #Pruebo volar duplicados, solo cambia el producto. Si el producto no importa
  #entonces volar duplicados no deberia importar. Obviamente vuelo el producto en el que
  #quede tambien.
  df.drop_duplicates('Opportunity_Name',inplace=True)
  df.drop(columns=['Product_Name','Product_Family','Opportunity_Name'],inplace=True)

  return df

In [3]:
# Metodo que pasa de DataFrame de Pandas a un DataSet de TensorFlow
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Stage') #Retorna la columna Stage, eliminandolo simultaneamente del DataFrame. 'Stage' seria nuestra columna 'target', es decir, lo que queremos predecir.
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels)) #Crea un DataSet cuyos elementos son slices de los tensores pasados a la funcion. Ver documentacion oficial para comprender bien.
  #En pocas palabras, le pasamos las columnas con los datos como diccionario (estilo 'columna':[dato1,dato2,dato3]) y una lista con los resultados estilo [resu1,resu2,resu3].
  #Se genera un DataSet del estilo [('columna':[dato1,dato2,dato3], [resu1, resu2, resu3])].
  #En realidad 'columna' es una lista de todas las columnas con sus correspondientes datos, pero se entiende la idea creo.
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe)) #Al tener el buffer_size del mismo tamanio que la cantidad de datos del dataset, tenemos perfect shuffling (ver documentacion para comprender).
    #Basicamente mezlcamos el dataset para que luego los batches que se armen contengan distintos elementos si lo entrenamos distintas veces.
  ds = ds.batch(batch_size) #Arma batches de tamanio batch_size entre elementos consecutivos del DataSet.
  return ds

In [4]:
#Arma las features, asignando las columnas del DataFrame segun corresponda al tipo de feature
#(numerico, categorico, etc). Entiendase por feature a las columnas del DataFrame.
def set_up_feature_columns(dataframe, numeric_columns, indicator_columns, bucket_columns):
  features = []

  #numeric features
  for column_name in numeric_columns:
    features.append(feature_column.numeric_column(column_name))

  #bucket features
  boundaries = [] #En principio este boundary es solo para la columna 'Delta Time'. Ver de como generalizar.
  for i in range(38):
    boundaries.append(i*5.0)

  for column_name in bucket_columns:
    range_column = feature_column.numeric_column(column_name)
    bukect_column = feature_column.bucketized_column(range_column, boundaries)
    features.append(bukect_column)

  #indicator_columns (one-hot value vector, para aquellas columnas categoricas de pocas opciones)
  for column_name in indicator_columns:
    categorical_column = feature_column.categorical_column_with_vocabulary_list(
                                          column_name, dataframe[column_name].unique())
    indicator_column = feature_column.indicator_column(categorical_column)
    features.append(indicator_column)

  return features

Preparamos los features para el modelo, es decir, seteamos cada una de las columnas que vayamos a utilizar del DataFrame. Luego generamos el DataSet en base al DataFrame para darselo como input al modelo.

In [5]:
  #Columnas que consideramos numericas
  numeric_columns = ['Pricing_Delivery_Terms_Quote_Appr','Pricing_Delivery_Terms_Approved',
                      'Bureaucratic_Code_0_Approval','Bureaucratic_Code_0_Approved',
                      'ASP_converted','TRF','Total_Taxable_Amount']

  #Columnas que consideramos clasificatorias con rango numerico
  bucket_columns = ['Delta_Time']

  #Columnas que consideramos categoricas de pocos valores posibles
  indicator_columns = ['Region','Bureaucratic_Code','Source',
                        'Account_Owner','Account_Type',
                        'Opportunity_Type','Quote_Type','Delivery_Terms',
                        'Product_Type','Size','Territory','Billing_Country',
                        'Account_Name','Opportunity_Owner',
                        'Brand','Product_Category_B','Last_Modified_By'
                        ]#'Product_Family', 'Product_Name', 'Opportunity_Name'

  df = pd.read_csv('/content/Train_TP2_Datos_2020-2C.csv')
  df = preprocess_dataframe(df)

  features = set_up_feature_columns(df,numeric_columns,indicator_columns,bucket_columns)

  #Separamos el DataFrame en uno de pruebo y el de entrenamiento. TODO: Ver el de validacion
  df_test = df.head(200)
  df.drop(df.head(200).index, inplace=True)
  df_validation = df.tail(200)
  df.drop(df.tail(200).index, inplace=True)

  feature_layer = tf.keras.layers.DenseFeatures(features)
  ds = df_to_dataset(df,batch_size=56, shuffle=False)
  ds_test = df_to_dataset(df_test, batch_size=56, shuffle=False)
  ds_validation = df_to_dataset(df_validation, batch_size=56, shuffle=False)

  df.head(5)

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:4582: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,Region,Territory,Pricing_Delivery_Terms_Quote_Appr,Pricing_Delivery_Terms_Approved,Bureaucratic_Code_0_Approval,Bureaucratic_Code_0_Approved,Bureaucratic_Code,Source,Billing_Country,Account_Name,Account_Owner,Opportunity_Owner,Account_Type,Opportunity_Type,Quote_Type,Delivery_Terms,Brand,Product_Type,Size,Product_Category_B,Last_Modified_By,ASP_converted,TRF,Total_Taxable_Amount,Stage,Delta_Time
447,EMEA,United Kingdom,1,0,0,0,Bureaucratic_Code_5,Source_7,United Kingdom,Account_Name_74,Person_Name_18,Person_Name_39,Account_Type_2,Opportunity_Type_1,Non Binding,Delivery_Terms_2,None,None,None,None,Person_Name_47,0.211572,-0.191691,-0.183942,1,7.0
448,EMEA,United Kingdom,1,0,0,0,Bureaucratic_Code_5,Source_7,United Kingdom,Account_Name_74,Person_Name_18,Person_Name_39,Account_Type_2,Opportunity_Type_1,Non Binding,Delivery_Terms_2,None,None,None,None,Person_Name_47,0.211572,-0.191691,-0.182350,1,6.0
449,Americas,NE America,1,1,0,0,Bureaucratic_Code_4,None,United States,Account_Name_749,Person_Name_3,Person_Name_3,Account_Type_0,Opportunity_Type_7,Non Binding,Delivery_Terms_4,None,None,None,None,Person_Name_47,0.278886,-0.109641,-0.198273,0,2.0
450,APAC,India,1,1,0,0,Bureaucratic_Code_4,Source_13,Sri Lanka,Account_Name_828,Person_Name_49,Person_Name_49,Account_Type_2,Opportunity_Type_1,Non Binding,Delivery_Terms_1,None,None,None,None,Person_Name_47,0.167092,-0.191691,-0.175632,1,14.0
451,EMEA,Italy,0,0,0,0,Bureaucratic_Code_4,Source_9,Italy,Account_Name_1000,Person_Name_23,Person_Name_23,Account_Type_0,Opportunity_Type_1,Non Binding,Delivery_Terms_2,None,None,None,None,Person_Name_47,0.219017,-0.191691,-0.186217,0,14.0


Creamos y compilamos el modelo. En esta seccion se tunean las propiedades del modelo.

In [6]:
model = tf.keras.Sequential([
  feature_layer,
  tf.keras.layers.Dropout(0.3),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

In [7]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

Entrenamos el modelo

In [13]:
#Ignoren el WARNING, esta en la documentacion tambien. Nadie le da bola en StackOverflow xd.
model.fit(ds, validation_data=ds_validation, epochs=60)
model.summary()

Epoch 1/60
168/168 [==============================] - 1s 6ms/step - loss: 0.3968 - accuracy: 0.8198 - val_loss: 0.4713 - val_accuracy: 0.7850
Epoch 2/60
168/168 [==============================] - 1s 6ms/step - loss: 0.3911 - accuracy: 0.8222 - val_loss: 0.4716 - val_accuracy: 0.7850
Epoch 3/60
168/168 [==============================] - 1s 7ms/step - loss: 0.3881 - accuracy: 0.8265 - val_loss: 0.4687 - val_accuracy: 0.7850
Epoch 4/60
168/168 [==============================] - 1s 6ms/step - loss: 0.3942 - accuracy: 0.8232 - val_loss: 0.4698 - val_accuracy: 0.7850
Epoch 5/60
168/168 [==============================] - 1s 6ms/step - loss: 0.3933 - accuracy: 0.8200 - val_loss: 0.4707 - val_accuracy: 0.7850
Epoch 6/60
168/168 [==============================] - 1s 7ms/step - loss: 0.3938 - accuracy: 0.8191 - val_loss: 0.4683 - val_accuracy: 0.7900
Epoch 7/60
168/168 [==============================] - 1s 6ms/step - loss: 0.3913 - accuracy: 0.8256 - val_loss: 0.4674 - val_accuracy: 0.7900
Epoch 

Evaluamos el modelo

In [15]:
model.evaluate(ds_test)

4/4 [==============================] - 0s 6ms/step - loss: 0.4045 - accuracy: 0.8050


[0.4045030474662781, 0.8050000071525574]

Escribimos las predicciones

In [16]:
frio_test_df = pd.read_csv('/content/Test_TP2_Datos_2020-2C.csv')
frio_test_df['Stage'] = 'Closed Won' #Esto esta solo para que funque todo, no lo uso. No se bien como armarlo sin los labels de Stage. TODO: Averiguar como es!
aux_df = frio_test_df[['Opportunity_ID']] #Esta columna la vuela el preprocesado sino
frio_test_df = preprocess_dataframe(frio_test_df)
frio_test_ds = df_to_dataset(frio_test_df, shuffle=False, batch_size=56)
predictions = model.predict(frio_test_ds)

aux_df.drop_duplicates(subset='Opportunity_ID', inplace=True) #Lo hacia el preprocesado pero es verdad que lo copie antes a este xd, perdon Agus, paja de dejarlo lindo.
aux_df['Target'] = predictions

aux_df.to_csv('prediccionesFrioFrio.csv', index=False)